### Question - 9
A cyber security agent wants to check the Microsoft Malware so need he came
to you as a Machine learning Engineering with Data, You need to find the Malware
using a supervised algorithm and you need to find the accuracy of the model.

In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split


In [2]:
import zipfile

# Unzip the file
with zipfile.ZipFile("/content/drive/MyDrive/ML/INTERMEDIATE/ML_INT_Q9/train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

In [3]:
file = open("/content/train.csv", encoding='utf-8')
n = sum(1 for line in file) - 1
s = 200000
skip = sorted(random.sample(range(1, n+1), n-s))
chunk = pd.read_csv("/content/train.csv", low_memory=False, chunksize=10000, skiprows=skip, )
df = pd.concat(chunk)

In [4]:
df

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,00001f26e9e5775277d6231fc6ac9e70,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1372.0,0,7.0,0,NaN,36429.0,...,51601.0,1,NaN,0.0,0,0,0.0,1.0,3.0,0
1,0000c5223307907251345099bfa89315,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1613.0,0,7.0,0,NaN,53447.0,...,20106.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,00010a37c34ebfb67cf79d7366cf3f56,win8defender,1.1.15100.1,4.9.10586.0,1.273.802.0,0,7.0,0,NaN,53447.0,...,33144.0,0,0.0,0.0,0,0,0.0,0.0,10.0,0
3,00013e5ab157f209e3a4dd836dd108ae,win8defender,1.1.15200.1,4.18.1807.18075,1.275.365.0,0,7.0,0,NaN,53447.0,...,61462.0,1,0.0,0.0,1,0,0.0,1.0,11.0,0
4,0002b528f6cc0790138f08473045a66d,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1094.0,0,7.0,0,NaN,53447.0,...,6876.0,1,NaN,0.0,1,0,0.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,fffeac6c0339d5982bbfcc31adc2507a,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1733.0,0,7.0,0,NaN,23657.0,...,12382.0,0,0.0,0.0,1,0,0.0,0.0,11.0,1
199996,fffef25a34f8e78132cad3cdfc89b720,win8defender,1.1.15200.1,4.13.17134.228,1.275.1798.0,0,7.0,0,NaN,47238.0,...,70177.0,0,0.0,0.0,0,1,0.0,1.0,6.0,0
199997,ffff0422f4d2986fdb33ac05a6ee796d,win8defender,1.1.15200.1,4.18.1806.18062,1.275.974.0,0,7.0,0,NaN,53447.0,...,13909.0,1,NaN,0.0,0,0,0.0,0.0,15.0,1
199998,ffff887f352b73a5be59a08229902d20,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1120.0,0,7.0,0,NaN,46669.0,...,33111.0,0,0.0,0.0,0,0,0.0,0.0,10.0,0


In [5]:
df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,200000.000000,199256.000000,200000.000000,9726.000000,199171.000000,199171.000000,199171.000000,200000.000000,200000.000000,192710.000000,...,196334.000000,200000.000000,72854.0,199668.000000,200000.000000,200000.000000,198361.000000,193156.000000,193156.000000,200000.000000
mean,0.000025,6.840180,0.017865,1673.077627,47834.690527,1.326825,1.020816,0.987785,108.096335,81211.356105,...,33018.469975,0.486905,0.0,0.006626,0.125095,0.037835,0.058046,0.283191,7.882370,0.500150
std,0.005000,1.045077,0.132461,1004.442154,14032.857788,0.523772,0.167316,0.109845,63.133863,48858.257236,...,21201.708348,0.499830,0.0,0.081130,0.330827,0.190798,0.233830,0.450549,4.545818,0.500001
min,0.000000,0.000000,0.000000,1.000000,39.000000,1.000000,0.000000,0.000000,1.000000,5.000000,...,22.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,7.000000,0.000000,788.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,36825.000000,...,13156.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.000000,7.000000,0.000000,1632.000000,53447.000000,1.000000,1.000000,1.000000,97.000000,82373.000000,...,33066.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000
75%,0.000000,7.000000,0.000000,2388.000000,53447.000000,2.000000,1.000000,1.000000,162.000000,122977.250000,...,52422.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,11.000000,1.000000
max,1.000000,35.000000,1.000000,3204.000000,70486.000000,6.000000,4.000000,1.000000,222.000000,167955.000000,...,72094.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 83 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   MachineIdentifier                                  200000 non-null  object 
 1   ProductName                                        200000 non-null  object 
 2   EngineVersion                                      200000 non-null  object 
 3   AppVersion                                         200000 non-null  object 
 4   AvSigVersion                                       200000 non-null  object 
 5   IsBeta                                             200000 non-null  int64  
 6   RtpStateBitfield                                   199256 non-null  float64
 7   IsSxsPassiveMode                                   200000 non-null  int64  
 8   DefaultBrowsersIdentifier                          9726 non-null    float6

In [7]:
df.columns

Index(['MachineIdentifier', 'ProductName', 'EngineVersion', 'AppVersion',
       'AvSigVersion', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode',
       'DefaultBrowsersIdentifier', 'AVProductStatesIdentifier',
       'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm',
       'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease',
       'OsBuildLab', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'PuaMode',
       'SMode', 'IeVerIdentifier', 'SmartScreen', 'Firewall', 'UacLuaenable',
       'Census_MDC2FormFactor', 'Census_DeviceFamily',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_ProcessorClass',
       'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
       'Census_

In [8]:
df.dropna(axis=1, thresh=120000, inplace=True)

In [9]:
#Correlation
correlation_matrix = df.corr()
threshold = 0.8 
highly_correlated_columns = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            # Adding both columns to the set of highly correlated columns
            col_i = correlation_matrix.columns[i]
            col_j = correlation_matrix.columns[j]
            highly_correlated_columns.add(col_i)
            highly_correlated_columns.add(col_j)

<ipython-input-9-23d8d6e8a477>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


In [10]:
df = df.drop(columns=highly_correlated_columns)


In [11]:
le = LabelEncoder()

for i in df.columns:
    if df[i].dtype == "object":
        df[i] = le.fit_transform(df[i])

In [12]:
#Splitting the dataset
df.replace(np.nan, 0, inplace=True)

X = df.drop("HasDetections", axis=1)
y = df['HasDetections']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [13]:
#Training the data and evaluation

model = lgb.LGBMClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_val)

#Calculating accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.6424


In [14]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.65      0.65     20061
           1       0.64      0.64      0.64     19939

    accuracy                           0.64     40000
   macro avg       0.64      0.64      0.64     40000
weighted avg       0.64      0.64      0.64     40000

